# Target and Feature Columns

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data_loc = 'data'

In [4]:
df_og = pd.read_pickle(f"{data_loc}/ml_dataset.pkl")

In [5]:
cols_ctrs = [col for col in df_og.columns if (len(col)==2) | (col=='Lebanon')]
cols_other = ['id']+[col for col in df_og.columns if col not in cols_ctrs]
df_ctrs = df_og[cols_ctrs]

In [6]:
df = df_og[cols_other]

In [7]:
df2 = df_og.dropna(subset=['ethnicity'])

In [8]:
df2 = df2[cols_other].dropna(subset=['imdb_votes'])

In [9]:
unwanted_cols = ['id', 'imdb_score', 'sentiment_t', 'sentiment', ]

In [10]:
target_col = 'imdb_score'
feature_cols = df_og.select_dtypes([float, int]).columns
feature_cols = feature_cols.drop(['imdb_score','tmdb_score','imdb_votes','tmdb_popularity', 'potential_seasons_left', 'recommended_min_age_viewership'])

In [11]:
feature_cols

Index(['release_year', 'runtime', 'seasons', 'action', 'animation', 'comedy',
       'crime', 'documentation', 'drama', 'european',
       ...
       'UY', 'VA', 'VE', 'VN', 'XX', 'ZA', 'ZW', 'sentiment_score',
       'potential_seasons', 'sentiment_score_t'],
      dtype='object', length=132)

# ML Setup

In [12]:
df_og = df_og.dropna(subset=['sentiment_score_t','sentiment_score', 'imdb_score'])
X = df_og[feature_cols]
y = df_og[target_col]

In [13]:
X.isna().sum()[X.isna().sum()>0]

Series([], dtype: int64)

No NaNs confirmed

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

# Ensemble Methods
from sklearn.ensemble import RandomForestRegressor 
from sklearn.ensemble import GradientBoostingRegressor # fits regression tree on neg gradient of loss functions
from sklearn.ensemble import AdaBoostRegressor # fit weak estimators on repeatedly modified versions of data, weigh them
from sklearn.ensemble import VotingRegressor # combine equally well performing models to balance out weaknesses
from sklearn.ensemble import StackingRegressor # uses output of individual estimators as input for new model

In [16]:
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=69, shuffle=True, test_size=0.2)

## Testing with Regression Models

In [18]:
def train_test_predict_score(model):
    m = model()
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    print(f"""Results:
    MSE: {mse}
    RMSE: {rmse}
    R2: {r2}
    """)

In [72]:
train_test_predict_score(LinearRegression)

Results:
    MSE: 0.8630015856966544
    RMSE: 0.9289787864621315
    R2: 0.2914836914667477
    


In [74]:
train_test_predict_score(SVR)

Results:
    MSE: 1.1879713002609649
    RMSE: 1.089940961823605
    R2: 0.02468656575539152
    


In [75]:
train_test_predict_score(DecisionTreeRegressor)

Results:
    MSE: 0.020687606691703653
    RMSE: 0.14383186952724925
    R2: 0.983015666519591
    


In [17]:
train_test_predict_score(RandomForestRegressor)

Results:
    MSE: 0.016375600136565378
    RMSE: 0.1279671838268131
    R2: 0.9865557839625405
    


## Random Forest Regressor and GridSearch

In [19]:
from sklearn.feature_selection import VarianceThreshold # Feature selector
from sklearn.pipeline import Pipeline # For setting up pipeline
# Various pre-processing steps
from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler, PowerTransformer, MaxAbsScaler
from sklearn.model_selection import GridSearchCV # For optimization

In [20]:
pipe = Pipeline([
#    ('pca', PCA(n_components = 4)),
    ('scaler', StandardScaler()),
    ('selector', VarianceThreshold()),
    ('regressor', RandomForestRegressor())
], verbose=True)
pipe.fit(X_train, y_train)
train_score = pipe.score(X_train, y_train)
test_score = pipe.score(X_test, y_test)

print(f"Training set score: {train_score}")
print(f"Testing set score: {test_score}")

[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.2s
[Pipeline] .......... (step 2 of 3) Processing selector, total=   0.1s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=  35.6s
Training set score: 0.9982066723797783
Testing set score: 0.9869010848185205


Number of trees, t
Size of Sample, n (Total - N)
Number of variable to sample at each node, m (Total - M)

In [22]:
parameters = {
    'scaler': [
        StandardScaler(), MinMaxScaler(),
        Normalizer(), MaxAbsScaler()
    ],
    'selector__threshold': [0, 0.1],
    'regressor__n_estimators': [100, 500],
    'regressor__min_samples_split': [2, 10],
    'regressor__oob_score': [True],
    'regressor__max_samples': [29289, 29289/3]
}

In [ ]:
grid = GridSearchCV(pipe, parameters, cv=2).fit(X_train, y_train)
 
print('Training set score: ' + str(grid.score(X_train, y_train)))
print('Test set score: ' + str(grid.score(X_test, y_test)))

In [24]:
train_score = grid.score(X_train, y_train)
test_score = grid.score(X_test, y_test)

In [27]:
print(f'Training set score: {train_score:.2f}')
print(f'Test set score: {test_score:.2f}')

Training set score: 0.99
Test set score: 0.98
